<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Привет! Поздравляю тебя с первым твоим проектом и спасибо, что вовремя сдал задание:) Ты проделал большую работу. Далее в файле мои коммефнтарии ты сможешь найти в ячейках, аналогичных данной ( если рамки комментария зелёные - всё сделано правильно; жёлтые - есть замечания, но не критично; красные - нужно переделать). Не удаляй эти комментарии и постарайся учесть их в ходе выполнения проекта. 

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
#импортируем необходимые модули и библиотеки
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
#откроем файл с данными и изучим общую информацию
df = pd.read_csv('/datasets/data.csv')
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Таблица с данными состоит из 21525 строк. Сразу же обращаю внимание на пропущенные значения в столбцах 'days_employed' и 'total_income'.

Среди данных в столбце 'days_employed' можно заметить отрицательные значения. К тому же, было бы неплохо привести данные столбцов 'days_employed' и 'total_income' к целочисленному значению для ускорения работы с данными и удобства восприятия информации.

Далее замечаю, что данные столбца 'education' записаны в разном регистре.
Наконец, присмотревшись к данным столбца 'purpose', обращаю внимание на многообразие формулировок целей кредита: например, простую "покупку жилья" клиенты банка описывают самыми разными словами, это нужно будет исправить.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Хорошее начало, так же, для большего понимания данных можно было бы использовать метод `describe`


### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
#ищем пропущенные значения
df['children'].value_counts()
df['education'].value_counts()
df['gender'].value_counts()
df['family_status'].value_counts()
df['purpose'].value_counts()
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
#избавляемся от пропусков в 'total_income' (заполним пропущенные значения медианными для данных типов занятости)
total_income_median = df.groupby('income_type')['total_income'].median()
df[df['total_income'].isnull()]['income_type'].value_counts()
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'сотрудник'), 'total_income'] = total_income_median[6]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'компаньон'), 'total_income'] = total_income_median[3]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'пенсионер'), 'total_income'] = total_income_median[4]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'госслужащий'), 'total_income'] = total_income_median[2]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'предприниматель'), 'total_income'] = total_income_median[5]

In [5]:
#избавляемся от артефакта в столбцах 'gender' и 'children'(удаляем). Осуществляем проверку
df = df[df['gender'] != 'XNA']
df = df[df['children'] != 20]
df = df[df['children'] != -1]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21401 entries, 0 to 21524
Data columns (total 12 columns):
children            21401 non-null int64
days_employed       19239 non-null float64
dob_years           21401 non-null int64
education           21401 non-null object
education_id        21401 non-null int64
family_status       21401 non-null object
family_status_id    21401 non-null int64
gender              21401 non-null object
income_type         21401 non-null object
debt                21401 non-null int64
total_income        21401 non-null float64
purpose             21401 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Вывод

Для обработки пропусков я использовал метод isnull() в случае числовых данных - это очень быстрый и простой способ. А в случае текстовых - оценил их правильность при помощи value_counts(): в отличие от unique() при нахождении артефактов он даст понять не только их значение, но и количество.

Поиск выявил пропуски в столбцах 'days_employed' и 'total_income' (значения NaN), а также артефакт ('XNA') в одной строчке столбца 'gender'. Отмечу сразу, что я решил не избавляться от пропусков в 'days_employed', потому что значения этого столбца не повлияют на результат исследования. Однако, нужно сообщить разработчикам об ошибках в данных этого столбца: здесь и отрицательные значения, и нереальные величины (например, стаж 340266.072047 дней - более 900 лет!), и упомянутые выше пропуски.

Для заполнения пропусков в столбце 'total_income' я решил рассчитать медиану дохода среди клиентов банка с тем же типом занятости. Отмечу, что все пропуски  в 'days_employed' и 'total_income' находятся на одних и тех же строчках, что наводит на мысль об ошибке при выгрузке данных, ведь среди отобранных типов занятости  нет "безработного".

Заполнить пропуск в столбце 'gender' невозможно: в таблице нет данных, которые косвенно могут указать на пол клиента (имя, фамилия и т.п.), поэтому можно либо удалить строчку с этой записью, либо оставить как есть. Я решил удалить, ведь в данном случае удаление одной строчки не окажет на результат значительного эффекта.

Кроме того, на данном этапе мной была замечена ошибка в данных столбца 'children'. Часть данных содержит значения, явно выбивающиеся из общей массы.: это либо отрицательные значения (-1), либо слишком большие(20). Несмотря на то, что эти данные не являются пропуском, оставлять как есть нельзя: выводы задачи напрямую зависят от числа детей в семье, поэтому я решил удалить эти строки.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Пропуски заполнены, молодец)

### Замена типа данных

In [6]:
#изменяем тип данных в столбце с доходом. Осуществляем проверку
df['total_income'] = df['total_income'].astype('int')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

Для удобства расчетов и восприятия информации я изменил тип данных в столбце 'total_income' с вещественного на целочисленный. Для этого я использовал метод astype(), выбранный мной из-за того, что метод to_numeric() здесь не подошел бы, ведь данные в этом столбце уже являются числами.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Замена типа данных выполнена верно

### Обработка дубликатов

In [7]:
#проверяем количество дубликатов
df.duplicated().sum()

54

In [8]:
#приводим значения столбца 'education' к нижнему регистру и осуществляем прмежуточную проверку
df['education'] = df['education'].str.lower()
df['education'].value_counts()
df.duplicated().sum()

71

In [9]:
#удаляем дублированные строчки и сразу нумеруем заново. Осуществляем проверку
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

### Вывод

Выявить вручную дубликаты в таблице из 21+ тысячи строк довольно сложно, поэтому мной был применен метод duplicated() для поиска дублированных строк. Изначально их число было равно 54. После этого я обратил внимание на замеченную мной ранее разницу регистра у записей в столбце 'education' и решил применить метод lower() для решения этой проблемы. Проверка, выполненная после приведения всех записей об образовании "к общему знаменателю", выявила, что число дублированных строк выросло и теперь составляет 71. Теперь дублированные строки можно удалять, применив метод drop_duplicates и не забыв при этом заново пронумеровать строчки таблицы.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Дубликаты найдены и обработаны верно,отлично


### Лемматизация

In [10]:
#на основе столбца purposes при помощи лемматизации создадим словарь целей кредита
purposes = ' '.join(df['purpose'].unique())
m = Mystem()
lemmas = m.lemmatize(purposes)
print(Counter(lemmas))
purpose_dict = ['недвижимость', 'автомобиль', 'образование', 'свадьба', 'жилье']

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


In [11]:
#создадим функцию, находящую значение в созданном нами словаре на основе данных столбца 'purpose'.
def find_purpose(df):
    for purpose in purpose_dict:
        if purpose in m.lemmatize(df):
            return purpose

In [12]:
#Выполним функцию и осуществим проверку   
df['purpose_lemmatized'] = df['purpose'].apply(find_purpose)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmatized
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


### Вывод

Для решения поставленной задачи необходимо было привести данные столбца 'purpose' к удобному для работы виду: получить обобщенные цели кредита. Поскольку я имел дело с множеством разных значений, я решил, что будет целесообразно не работать со стеммером, а лемматизировать значения колонки 'purpose'. При помощи коллекции Counter лемматизированные значения удалось отобрать вручную и создать словарь целей кредита. Создав и применив функцию, я получил новый столбец, в котором всего 5 релевантных значений, что гораздо удобнее для анализа.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>


Лемматизация проведена верно

### Категоризация данных

In [13]:
#определяем медианный доход и границы категорий доходов
median_income = df['total_income'].median()
low_income_border = median_income * 7 / 10
high_income_border = median_income * 2

In [14]:
#создаем функцию для опрделения категории дохода на основе данных столбца 'total_income'
def income_categorized(total_income):
    if total_income < low_income_border:
        return 'низкий доход'
    elif low_income_border <= total_income < high_income_border:
        return 'средний доход'
    else:
        return 'высокий доход'

In [15]:
#Выполним функцию и осуществим проверку
df['total_income_categorized'] = df['total_income'].apply(income_categorized)
print(df['total_income_categorized'].value_counts())
df.head()

средний доход    15164
низкий доход      4420
высокий доход     1746
Name: total_income_categorized, dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmatized,total_income_categorized
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,средний доход
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний доход
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,средний доход
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,средний доход
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний доход


### Вывод

Для ответа на вопросы ислледования почти все готово, осталось лишь категоризировать клиентов банка по уровню дохода. Чтобы разбить доходы клиентов на три категории я использовал относительный подход "медианного дохода": согласно нему к низкому доходу причисляются люди с доходом ниже 0,7 медианного, к высокому - люди с доходом более двух медианных, а к среднему, соответсвенно, от 0,7 до 2 медианных доходов (информация из презентации подготовленной Высшей школой экономики, 2019г.). Высчитав медианный доход и на его основе установив границы категорий, я создал функцию, после применения которой все клиенты банка были отнесены к трех категориям согласно доходу.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Категоризация проведена верно

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [16]:
#создадим сводную таблицу на основе данных столбцов 'children' и 'debt'
children_debt = pd.pivot_table(df, index = 'children', columns = 'debt', values = 'family_status_id', aggfunc = 'count')
#отсортируем результат в порядке убывания
children_debt.sort_values(by=0, ascending=False)

debt,0,1
children,,
0,13027.0,1063.0
1,4364.0,444.0
2,1858.0,194.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


### Вывод

Взглянув на сводную таблицу, можно сказать что во всех категориях соблюдается примерное соотношение возвратов к долгам: 10 к 1. Однако, клиенты банка, не имеющие детей, отдадут кредит с большей вероятностью, чем остальные, а к тем, у кого 1,2 или 4 ребенка, нужно присмотреться внимательнее. Это можно объяснить тем, что дети требуют дополнительных расходов, на которые у родителей не всегда находятся деньги.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [17]:
#создадим сводную таблицу на основе данных столбцов 'family_status' и 'debt'
family_status_debt = pd.pivot_table(df, index = 'family_status', columns = 'debt', values = 'family_status_id', aggfunc = 'count')
#отсортируем результат в порядке убывания
family_status_debt.sort_values(by=0, ascending=False)

debt,0,1
family_status,,
женат / замужем,11334,927
гражданский брак,3748,385
Не женат / не замужем,2523,273
в разводе,1105,84
вдовец / вдова,888,63


### Вывод

Сопоставив семейный статус с наличием долгов, можно сказать, что клиенты, находящиеся в официальных отношениях, а также те, кто состоял в них и косвенно указывает на это в анкете ("вдовец/вдова" и "в разводе", вместо "не женат/не замужем") с большей вероятностью закроют кредит в срок, чем те, кто находится в гражданском браке или холост. Можно сказать, что причастность к официальному браку положительно сказывается на сознательности должников.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [18]:
#создадим сводную таблицу на основе данных столбцов 'total_income_categorized' и 'debt'
income_debt = pd.pivot_table(df, index = 'total_income_categorized', columns = 'debt', values = 'family_status_id', aggfunc = 'count')
#отсортируем результат в порядке убывания
income_debt.sort_values(by=0, ascending=False)

debt,0,1
total_income_categorized,,
средний доход,13909,1255
низкий доход,4067,353
высокий доход,1622,124


### Вывод

Среди категорий по доходам наибольший процент должников можем наблюдать среди клиентов банка со средним уровнем дохода. Это вполне закономерно: в отличие от тех, кто имеет низкий доход и не обращается к кредитам без нужды из-за страха, а также тех, кто имеет высокий доход и использует кредиты как финансовый инструмент, "средний класс" порой чересчур закредитовывается.

- Как разные цели кредита влияют на его возврат в срок?

In [19]:
#создадим сводную таблицу на основе данных столбцов 'purpose_lemmatized' и 'debt'
purpose_debt = pd.pivot_table(df, index = 'purpose_lemmatized', columns = 'debt', values = 'family_status_id', aggfunc = 'count')
#отсортируем результат в порядке убывания
purpose_debt.sort_values(by=0, ascending=False)

debt,0,1
purpose_lemmatized,,
недвижимость,5841,472
жилье,4129,308
автомобиль,3879,400
образование,3619,369
свадьба,2130,183


### Вывод

Наконец, в зависимости от целей кредита, можно утверждать, что с меньшей вероятностью в срок будет возвращен автокредит и кредит на образование. Среди клиентов банка, взявших ипотеку или кредит на свадьбу, процент должников значительно меньше.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Все выводы верны, код написан правильно, радует, что используешь сводные таблицы)


### Шаг 4. Общий вывод

В результате исследования мы пришли к выводу о том, что наибольшей возможностью вернуть кредит в срок обладают люди, не имеющие детей. Кроме того, среди клиентов банка, состоящих в официальном браке, процент должников меньше. Значит, молодая семья без детей - сегмент, на который стоит обратить внимание. Возможно, стоит создать предложение по ипотеке для данной группы, ведь кредиты на недвижимость, как показали результаты анализа, обладают положительной характеристикой.

<div style="border:solid  orange  2px; padding: 20px"> <h1 style="color: orange ; margin-bottom:20px">Комментарий наставника</h1>

Обязательно надо писать подробный вывод из проведённого анализа, в котором отразить все полученные результаты + дать советы по использованию данного анализа


### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.

<div style="border:solid  green  2px; padding: 20px"> <h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>

#### Код

Всё отлично. Из того, что очень порадовало - соблюдена структура проекта, шаги из задания обозначены и выполнены последовательно, код написан аккуратно, используются комментарии к коду, быстро можно понять, какие операции выполняют сложные конструкции. В качестве совета предлагаю глубже изучить и начать чаще применять конструкцию try-except в решении задачи — это улучшит отказоустойчивость кода и обезопасит код от поломок в будущем, а так же изучить средства для построения изображений в Python для более глубокого понимания данных.
#### Выводы

У тебя отлично получается анализировать сложные данные, выдвигать корректные гипотезы и проверять свои выводы на возможность соответствия реальности. Видно глубокое понимание сути проведённого анализа. Было очень интересно проверять твой проект и следить за твоей мыслью, так держать!)


<div style="border:solid  green  2px; padding: 20px"> <h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>
Работа принята!) Очень рад, что у тебя всё получилось и надеюсь тебе понравится весь курс, будь усерден и у тебя всё получится! Удачи в будущих проектах)